<a href="https://colab.research.google.com/github/tusharvatsa32/Language-Models/blob/main/C_Bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tusharvatsa32/nn4nlp-code.git

Cloning into 'nn4nlp-code'...
remote: Enumerating objects: 545, done.
remote: Total 545 (delta 0), reused 0 (delta 0), pack-reused 545
Receiving objects: 100% (545/545), 11.92 MiB | 4.02 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [4]:
pip install dynet

     |████████████████████████████████| 4.5MB 11.7MB/s 


In [5]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [6]:
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                     # Softmax bias

In [7]:
def calc_scores(words):
  dy.renew_cg()
  cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
  W_sm_exp = dy.parameter(W_sm)
  b_sm_exp = dy.parameter(b_sm)
  return W_sm_exp * cbow + b_sm_exp

In [8]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))


The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
iter 0: train loss/sent=1.4667, time=0.24s
iter 0: test acc=0.3900
iter 1: train loss/sent=1.0834, time=0.32s
iter 1: test acc=0.4190
iter 2: train loss/sent=0.7721, time=0.33s
iter 2: test acc=0.4109
iter 3: train loss/sent=0.5347, time=0.31s
iter 3: test acc=0.3900
iter 4: train loss/sent=0.3671, time=0.33s
iter 4: test acc=0.3964
iter 5: train loss/sent=0.2517, time=0.32s
iter 5: test acc=0.3882
iter 6: train loss/sent=0.1786, time=0.32s
iter 6: test acc=0.3855
iter 7: train loss/sent=0.1309, time=0.34s
iter 7: test acc=0.3774
iter 8: train loss/sent=0.0983, time=0.32s
iter 8: test acc=0.3801
iter 9: train loss/sent=0.0717, time=0.33s
iter 9: test acc=0.3674
iter 10: train loss/sent=0.0567, time=0.33s
iter 10: test acc=0.3738
iter 11: train loss/sent=0.0436, time=0.32s
iter 11: test acc=0.3828
ite